## Importação da planilha BrazilianCities.xlsx 

#### Utilizando as bibliotecas pandas e psycopg2

In [1]:
# importa bibliotecas
import pandas as pd
import psycopg2
import psycopg2.extras as extras


In [2]:
# Variáveis de conexão
host = ''
dbName = 'bd_cidades_estados'
user = ''
pwd = ''

In [3]:
# Funcões

# Função para conexão com o banco de dados
def connect_database(host, dbName, user, pwd):
    """
    Função para conectar o banco de dados

    Parametros:
        host   : Instancia PostgreSQL
        dbName : banco de dados
        user   : usuário
        pwd    : senha do usuário
        
    Retornos:
        cur  : Utilize para conectar e executar os comandos SQL.
        conn : Conexão com o banco de dados.
    """
    # connect to default database
    conn = psycopg2.connect(f"host={host} dbname={dbName} user={user} password={pwd}")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    return cur, conn

# Função para criar as tabelas
def create_tables(stmt_create_tables):
    """
    Função para criar tabelas

    Parametros:
        cur  : Utilize para conectar e executar os comandos SQL.
        conn : Conexão com o banco de dados.
    """
    for stmt in stmt_create_tables:
        cur.execute(stmt)
        conn.commit()
    return


# Função genérica para inserir linhas
def insert_rows(df, insert_stmt):
    """
    Função genérica para inserir linhas em tabela
    
    Parametros:
        df          : Dataframe com os valores a serem inseridos
        insert_stmt : Comando de insert 
    """
    # Executa o loop para gerar as tuplas com os valores a serem inseridos
    for row in df.itertuples(index=False):
        # print(tuple(row))
        # Executa o insert
        cur.execute(insert_stmt, tuple(row))
    print("Dados inseridos com sucesso...")
    return

# função usando psycopg2.extras.execute_values() para inserir o dataframe
def insert_tb_cidade(df):    
    """
    Função usando psycopg2.extras.execute_values() para inserir o dataframe
    
    Parametros:
        df    : Dataframe com os valores a serem inseridos
    """   
    # Criação de uma lista de tupples a partir dos valores do dataframe
    tpls = [tuple(row) for row in df.itertuples(index=False)]
    
    # Comando de inserção
    sql = """ INSERT INTO tb_cidade (nome_cidade, id_estado) VALUES %s"""
    try:
        extras.execute_values(cur, sql, tpls)
        print("Dados inseridos usando execute_values() com sucesso...")
    except (Exception, psycopg2.DatabaseError) as err:
        show_psycopg2_exception(err)
        cursor.close()
    return


In [4]:
# Conexão com o banco de dados Postgre
cur, conn = connect_database(host, dbName, user, pwd)

In [5]:
# Comandos para criação das tabelas
stmt_create_tables = list()
stmt_create_tables.append('CREATE TABLE IF NOT EXISTS tb_estado (id_estado smallserial not null primary key, nome_estado varchar(100) not null, sigla_estado char(02) not null);')
stmt_create_tables.append('CREATE TABLE IF NOT EXISTS tb_cidade (id_cidade smallserial not null primary key, nome_cidade varchar(300) not null, id_estado smallint not null);')

In [6]:
# Cria as tabelas no banco de dados
create_tables(stmt_create_tables)

In [7]:
# Cria datraframe com a planilha principal
df = pd.read_excel('BrazilianCities.xlsx', sheet_name='BrazilianCities')

In [8]:
# Gera dataframe de estado
df_estado = df[['Estado', 'Sigla']].copy()

In [9]:
df_estado.shape

(5570, 2)

In [10]:
# Retira os estados repetidos
df_estado = df_estado.drop_duplicates(keep='last').copy()

In [14]:
df_estado.shape

(27, 2)

In [15]:
#Comando para truncar a tabela 
cur.execute('truncate table tb_estado restart identity')  

#Comando para inserir os registros
postgres_insert_query = """ INSERT INTO tb_estado (id_estado, nome_estado, sigla_estado) VALUES (DEFAULT,%s,%s)"""

#Chamada da função para inserção dos dados (linha a linha)
insert_rows(df_estado, postgres_insert_query)

Dados inseridos com sucesso...


In [16]:
# Select na tabela tb_estado para verificar as linhas inseridas
cur.execute('SELECT * FROM tb_estado')  
rows = cur.fetchall()
rows

[(1, 'DISTRITO FEDERAL', 'DF'),
 (2, 'RORAIMA', 'RR'),
 (3, 'SERGIPE', 'SE'),
 (4, 'AMAZONAS', 'AM'),
 (5, 'MATO GROSSO DO SUL', 'MS'),
 (6, 'ALAGOAS', 'AL'),
 (7, 'CEARÁ', 'CE'),
 (8, 'RIO GRANDE DO NORTE', 'RN'),
 (9, 'GOIÁS', 'GO'),
 (10, 'MATO GROSSO', 'MT'),
 (11, 'RONDÔNIA', 'RO'),
 (12, 'ESPÍRITO SANTO', 'ES'),
 (13, 'AMAPÁ', 'AP'),
 (14, 'RIO DE JANEIRO', 'RJ'),
 (15, 'PIAUÍ', 'PI'),
 (16, 'MINAS GERAIS', 'MG'),
 (17, 'TOCANTINS', 'TO'),
 (18, 'PARANÁ', 'PR'),
 (19, 'RIO GRANDE DO SUL', 'RS'),
 (20, 'ACRE', 'AC'),
 (21, 'PERNAMBUCO', 'PE'),
 (22, 'PARÁ', 'PA'),
 (23, 'BAHIA', 'BA'),
 (24, 'PARAÍBA', 'PB'),
 (25, 'SÃO PAULO', 'SP'),
 (26, 'MARANHÃO', 'MA'),
 (27, 'SANTA CATARINA', 'SC')]

In [17]:
# Cria o Dataframe com os estados a partir da leitura da tabela para carregar o id_estado
df_estadoSQL = pd.read_sql('SELECT * FROM tb_estado', conn);

In [18]:
df_estadoSQL

,id_estado,nome_estado,sigla_estado
0,1,DISTRITO FEDERAL,DF
1,2,RORAIMA,RR
2,3,SERGIPE,SE
3,4,AMAZONAS,AM
4,5,MATO GROSSO DO SUL,MS
5,6,ALAGOAS,AL
6,7,CEARÁ,CE
7,8,RIO GRANDE DO NORTE,RN
8,9,GOIÁS,GO
9,10,MATO GROSSO,MT


In [19]:
# Cria o dataframe df_cidade com o join do dataframe df (planilha) e dataframe df_estadoSQL 
# para referenciar o id_estado nas cidades
df_cidade = pd.merge(left=df, right=df_estadoSQL, left_on='Sigla', right_on='sigla_estado').copy()

In [21]:
# Processo incluindo em lote utilizando psycopg2.extras.execute_values() é mais performático
cur.execute('truncate table tb_cidade restart identity')  
insert_tb_cidade(df_cidade[['Cidade', 'id_estado']])

Dados inseridos usando execute_values() com sucesso...


In [31]:
# Select na tabela tb_cidade para contabilizar as cidades por estado
cur.execute("""
            select 	count(id_cidade) AS "Qtd_Cidades", 
                    e.nome_estado, 
                    e.sigla_estado 
            from 		tb_cidade c 
            inner join 	tb_estado e 
                    on	e.id_estado = c.id_estado  
            group by e.nome_estado, e.sigla_estado
            order by e.sigla_estado
            """)
rows = cur.fetchall()
rows

[(22, 'ACRE', 'AC'),
 (102, 'ALAGOAS', 'AL'),
 (62, 'AMAZONAS', 'AM'),
 (16, 'AMAPÁ', 'AP'),
 (417, 'BAHIA', 'BA'),
 (184, 'CEARÁ', 'CE'),
 (1, 'DISTRITO FEDERAL', 'DF'),
 (78, 'ESPÍRITO SANTO', 'ES'),
 (246, 'GOIÁS', 'GO'),
 (217, 'MARANHÃO', 'MA'),
 (853, 'MINAS GERAIS', 'MG'),
 (79, 'MATO GROSSO DO SUL', 'MS'),
 (141, 'MATO GROSSO', 'MT'),
 (144, 'PARÁ', 'PA'),
 (223, 'PARAÍBA', 'PB'),
 (185, 'PERNAMBUCO', 'PE'),
 (224, 'PIAUÍ', 'PI'),
 (399, 'PARANÁ', 'PR'),
 (92, 'RIO DE JANEIRO', 'RJ'),
 (167, 'RIO GRANDE DO NORTE', 'RN'),
 (52, 'RONDÔNIA', 'RO'),
 (15, 'RORAIMA', 'RR'),
 (497, 'RIO GRANDE DO SUL', 'RS'),
 (295, 'SANTA CATARINA', 'SC'),
 (75, 'SERGIPE', 'SE'),
 (645, 'SÃO PAULO', 'SP'),
 (139, 'TOCANTINS', 'TO')]

In [22]:
# Select na tabela tb_cidade para verificar as linhas inseridas
cur.execute('SELECT * FROM tb_cidade')  
rows = cur.fetchall()
rows

[(1, 'ABADIA DE GOIÁS', 9),
 (2, 'ABADIÂNIA', 9),
 (3, 'ACREÚNA', 9),
 (4, 'ADELÂNDIA', 9),
 (5, 'ÁGUA FRIA DE GOIÁS', 9),
 (6, 'ÁGUA LIMPA', 9),
 (7, 'ÁGUAS LINDAS DE GOIÁS', 9),
 (8, 'ALEXÂNIA', 9),
 (9, 'ALOÂNDIA', 9),
 (10, 'ALTO HORIZONTE', 9),
 (11, 'ALTO PARAÍSO DE GOIÁS', 9),
 (12, 'ALVORADA DO NORTE', 9),
 (13, 'AMARALINA', 9),
 (14, 'AMERICANO DO BRASIL', 9),
 (15, 'AMORINÓPOLIS', 9),
 (16, 'ANÁPOLIS', 9),
 (17, 'ANHANGUERA', 9),
 (18, 'ANICUNS', 9),
 (19, 'APARECIDA DE GOIÂNIA', 9),
 (20, 'APARECIDA DO RIO DOCE', 9),
 (21, 'APORÉ', 9),
 (22, 'ARAÇU', 9),
 (23, 'ARAGARÇAS', 9),
 (24, 'ARAGOIÂNIA', 9),
 (25, 'ARAGUAPAZ', 9),
 (26, 'ARENÓPOLIS', 9),
 (27, 'ARUANÃ', 9),
 (28, 'AURILÂNDIA', 9),
 (29, 'AVELINÓPOLIS', 9),
 (30, 'BALIZA', 9),
 (31, 'BARRO ALTO', 9),
 (32, 'BELA VISTA DE GOIÁS', 9),
 (33, 'BOM JARDIM DE GOIÁS', 9),
 (34, 'BOM JESUS DE GOIÁS', 9),
 (35, 'BONFINÓPOLIS', 9),
 (36, 'BONÓPOLIS', 9),
 (37, 'BRAZABRANTES', 9),
 (38, 'BRITÂNIA', 9),
 (39, 'BURITI ALEGRE', 9)

In [ ]:
##############################################################################################
#
# NÃO É NECESSÁRIO EXECUTAR ESSA CÉLULA, POIS O INSERT LINHA A LINHA DESSA É DEMORADO
#
# ESTA AQUI APENAS PARA EXPERIÊNCIA DIDÁTICA
#
##############################################################################################

# Processo incluindo linha a linha - é bem demorado (15 min = 5.000)
postgres_insert_query = """ INSERT INTO tb_cidade (id_cidade, nome_cidade, id_estado) VALUES (DEFAULT,%s,%s)"""
cur.execute('truncate table tb_cidade restart identity')  
insert_rows(df_cidade[['Cidade', 'id_estado']], postgres_insert_query)